In [1]:
import os
%pwd

'd:\\college_work\\4th year\\Sem7th\\Project\\OBE\\Paper- Blooms taxanomy\\Boolm-s-Level-Detection-A-MLOPS-Project\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\college_work\\4th year\\Sem7th\\Project\\OBE\\Paper- Blooms taxanomy\\Boolm-s-Level-Detection-A-MLOPS-Project'

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/ukantjadia/Boolm-s-Level-Detection-A-MLOPS-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="ukantjadia"
os.environ["MLFLOW_TRACKING_PASSWORD"]="cb7df2a64a9e62c2f3b5a892a1e0593545a8ea77" 

In [4]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path 
    test_data_path: Path 
    model_path: Path 
    metric_file_name: Path
    mlflow_URI: str 
    all_params: dict
    target_column: str
    training_columns: str

In [6]:
from blooms import logger
from blooms.utils.common import read_yaml, create_directories, save_json
from blooms.constants import *

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.NBMultinomial
        schema = self.schema.COLUMNS
        create_directories([config.root_dir])
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            all_params=params,
            mlflow_URI="https://dagshub.com/ukantjadia/Boolm-s-Level-Detection-A-MLOPS-Project.mlflow",
            target_column=schema.target_column,
            training_columns=schema.training_columns,
        )
        return model_evaluation_config

In [22]:
import pandas as pd
import os 
from sklearn.metrics import accuracy_score,classification_report,mean_absolute_error,mean_squared_error,r2_score
from urllib.parse import urlparse
import mlflow 
import mlflow.sklearn
import numpy as np   
import joblib 
from sklearn.preprocessing import LabelEncoder

In [25]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config = config 
        
    def eval_matrix(self,actual,pred):
        rmse = mean_squared_error(actual,pred)
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        acc = accuracy_score(actual,pred)
        report = classification_report(actual,pred)
        return rmse, mae, r2, acc, report
    
    def log_into_mlflow(self):
        encoder = LabelEncoder()
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(filename=self.config.model_path)
        
        test_x = test_data[self.config.training_columns].tolist()
        test_y = test_data[self.config.target_column].tolist()
        
        mlflow.set_registry_uri(self.config.mlflow_URI)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2, acc, report) = self.eval_matrix(encoder.fit_transform(test_y),encoder.fit_transform(predicted_qualities))
            scores = {"rmse":rmse,"mae":mae,"r2":r2,"accuracy":acc}
            save_json(path=Path(self.config.metric_file_name),data=scores)
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)
            mlflow.log_metric("accuracy",acc)
            
            if tracking_url_type_store != 'file':
                mlflow.sklearn.log_model(model,"model",registered_model_name="NB Multinomial")
            else:
                mlflow.sklearn.load_model(model,"model")

In [26]:
try: 
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-12-19 16:51:37: INFO: common: yaml file: config\config.yaml loaded succcessfully]
[2023-12-19 16:51:37: INFO: common: yaml file: params.yaml loaded succcessfully]
[2023-12-19 16:51:37: INFO: common: yaml file: schema.yaml loaded succcessfully]
[2023-12-19 16:51:37: INFO: common: created directory at : artifacts]
[2023-12-19 16:51:37: INFO: common: created directory at : artifacts/model_evaluation]


c:\Users\Ukant\anaconda3\envs\blooms\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Ukant\anaconda3\envs\blooms\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'NB Multinomial' already exists. Creating a new version of this model...
2023/12/19 16:51:54 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: NB Multinomial, version 2
Created version '2' of model 'NB Multinomial'